# Inspect Data

This example shows how to inspect the data of a loaded test system.

In [1]:
import andes
from andes.utils.paths import get_case

To show all the rows and columns, change the pandas configuration with

In [2]:
import pandas as pd

pd.options.display.max_columns = None
pd.options.display.max_rows = None

Let's load the Kundur's system.

## Load System from an ANDES XLSX File

The ANDES xlsx file is the best supported format. Other formats can be converted to the xlsx format.

See the link below for format conversion.
https://github.com/cuihantao/andes/blob/master/README.md#format-converter

In [3]:
ss = andes.run(get_case('kundur/kundur_full.xlsx'))

Working directory: "/Users/hcui7/repos/andes/examples"
Parsing input file "/Users/hcui7/repos/andes/andes/cases/kundur/kundur_full.xlsx"
Input file parsed in 0.0581 second.

-> Power flow calculation
Method: Full Newton-Raphson method
Power flow initialized.
0: |F(x)| = 14.9283   
1: |F(x)| = 3.60859   
2: |F(x)| = 0.170093  
3: |F(x)| = 0.00203827
4: |F(x)| = 3.76414e-07
Converged in 5 iterations in 0.0093 second.
Report saved to "kundur_full_out.txt" in 0.0018 second.
-> Single process finished in 0.1130 second.


## Load System from PSS/E RAW and DYR Files

Alternatively, ANDES supports loading systems from PSS/E RAW and DYR files.

The PSS/E v32 raw format is best supported.

Note that this feature is experimental. We try out best to support this format, but the compatibility is not guaranteed.


In [4]:
raw_path = get_case('kundur/kundur_full.raw')
dyr_path = get_case('kundur/kundur_full.dyr')

# the raw file is passed to the positional argument, while the dyr file is passed to `addfile`.
ss = andes.run(raw_path, addfile=dyr_path)

Working directory: "/Users/hcui7/repos/andes/examples"
Parsing input file "/Users/hcui7/repos/andes/andes/cases/kundur/kundur_full.raw"
  MODIFIED KUNDUR'S TWO-AREA TEST SYSTEM, DISTRIBUTED WITH ADNDES
  SEE THE BOOK "POWER SYSTEM STABILITY AND CONTROL" FOR ORIGINAL DATA
Input file parsed in 0.0039 second.
Parsing additional file "/Users/hcui7/repos/andes/andes/cases/kundur/kundur_full.dyr"
Experimental feature: PSS/E dyr support is incomplete.
Addfile parsed in 0.0306 second.

-> Power flow calculation
Method: Full Newton-Raphson method
Power flow initialized.
0: |F(x)| = 3.17584   
1: |F(x)| = 3.17627e-06
2: |F(x)| = 3.33598e-10
Converged in 3 iterations in 0.0058 second.
Report saved to "kundur_full_out.txt" in 0.0011 second.
-> Single process finished in 0.0741 second.


## Parameters

Parameters from the input file are stored in `cache.df_in` of the model. For `GENROU`, do

In [5]:
ss.GENROU.cache.df_in

,idx,u,name,bus,gen,coi,Sn,Vn,fn,D,M,ra,xl,xq,kp,kw,S10,S12,xd,xd1,xd2,xq1,xq2,Td10,Td20,Tq10,Tq20
uid,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,GENROU_0,1.0,GENROU_0,1,1,None,900.0,20.0,60.0,0.0,13.00,0.0,0.06,1.7,0.0,0.0,0.0,1.0,1.8,0.3,0.25,0.55,0.25,8.0,0.03,0.4,0.05
1,GENROU_1,1.0,GENROU_1,2,2,None,900.0,20.0,60.0,0.0,13.00,0.0,0.06,1.7,0.0,0.0,0.0,1.0,1.8,0.3,0.25,0.55,0.25,8.0,0.03,0.4,0.05
2,GENROU_2,1.0,GENROU_2,3,3,None,900.0,20.0,60.0,0.0,12.35,0.0,0.06,1.7,0.0,0.0,0.0,1.0,1.8,0.3,0.25,0.55,0.25,8.0,0.03,0.4,0.05
3,GENROU_3,1.0,GENROU_3,4,4,None,900.0,20.0,60.0,0.0,12.35,0.0,0.06,1.7,0.0,0.0,0.0,1.0,1.8,0.3,0.25,0.55,0.25,8.0,0.03,0.4,0.05


Parameters will be **converted** to system per-unit base after loading. To inspect the converted parameters, check the `cache.df` parameter.

In [6]:
ss.GENROU.cache.df

,idx,u,name,bus,gen,coi,Sn,Vn,fn,D,M,ra,xl,xq,kp,kw,S10,S12,xd,xd1,xd2,xq1,xq2,Td10,Td20,Tq10,Tq20
uid,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,GENROU_0,1.0,GENROU_0,1,1,None,900.0,20.0,60.0,0.0,117.00,0.0,0.006667,0.188889,0.0,0.0,0.0,1.0,0.2,0.033333,0.027778,0.061111,0.027778,8.0,0.03,0.4,0.05
1,GENROU_1,1.0,GENROU_1,2,2,None,900.0,20.0,60.0,0.0,117.00,0.0,0.006667,0.188889,0.0,0.0,0.0,1.0,0.2,0.033333,0.027778,0.061111,0.027778,8.0,0.03,0.4,0.05
2,GENROU_2,1.0,GENROU_2,3,3,None,900.0,20.0,60.0,0.0,111.15,0.0,0.006667,0.188889,0.0,0.0,0.0,1.0,0.2,0.033333,0.027778,0.061111,0.027778,8.0,0.03,0.4,0.05
3,GENROU_3,1.0,GENROU_3,4,4,None,900.0,20.0,60.0,0.0,111.15,0.0,0.006667,0.188889,0.0,0.0,0.0,1.0,0.2,0.033333,0.027778,0.061111,0.027778,8.0,0.03,0.4,0.05


### Refresh

If any parameter is done through `Model.alter()`, `cache` must to be refreshed to reflect the latest values.

In [ ]:
ss.GENROU.alter("M", "GENROU_0", 10)

In [ ]:
ss.GENROU.cache.refresh()

In [ ]:
ss.GENROU.cache.df_in

## Cleanup

If any parameter is done through `Model.alter()`, `cache` must to be refreshed to reflect the latest values.

In [8]:
ss.GENROU.alter("M", "GENROU_0", 10)

In [9]:
ss.GENROU.cache.refresh()

In [10]:
ss.GENROU.cache.df_in

,idx,u,name,bus,gen,coi,Sn,Vn,fn,D,M,ra,xl,xq,kp,kw,S10,S12,xd,xd1,xd2,xq1,xq2,Td10,Td20,Tq10,Tq20
uid,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,GENROU_0,1.0,GENROU_0,1,1,None,900.0,20.0,60.0,0.0,10.00,0.0,0.06,1.7,0.0,0.0,0.0,1.0,1.8,0.3,0.25,0.55,0.25,8.0,0.03,0.4,0.05
1,GENROU_1,1.0,GENROU_1,2,2,None,900.0,20.0,60.0,0.0,13.00,0.0,0.06,1.7,0.0,0.0,0.0,1.0,1.8,0.3,0.25,0.55,0.25,8.0,0.03,0.4,0.05
2,GENROU_2,1.0,GENROU_2,3,3,None,900.0,20.0,60.0,0.0,12.35,0.0,0.06,1.7,0.0,0.0,0.0,1.0,1.8,0.3,0.25,0.55,0.25,8.0,0.03,0.4,0.05
3,GENROU_3,1.0,GENROU_3,4,4,None,900.0,20.0,60.0,0.0,12.35,0.0,0.06,1.7,0.0,0.0,0.0,1.0,1.8,0.3,0.25,0.55,0.25,8.0,0.03,0.4,0.05


## Cleanup

In [7]:
!andes misc -C


    _           _         | Version 0.8.3.post20.dev0+gd48f2a7b
   /_\  _ _  __| |___ ___ | Python 3.7.1 on Darwin, 04/06/2020 05:40:44 PM
  / _ \| ' \/ _` / -_|_-< | 
 /_/ \_\_||_\__,_\___/__/ | This program comes with ABSOLUTELY NO WARRANTY.

<kundur_full_out.txt> removed.
